In [19]:
import pandas as pd
import numpy as np
import os
from scipy.cluster import hierarchy as hc
import matplotlib
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
from scipy import stats
import math
from sklearn.preprocessing import MinMaxScaler

%matplotlib inline
warnings.filterwarnings(action='ignore')

matplotlib.rcParams['font.family'] ='Malgun Gothic'
matplotlib.rcParams['axes.unicode_minus'] =False

# Load data

In [20]:
# dir_root = 'C:/Users/kkt71/OneDrive/바탕 화면/기업가치분석'
dir_root = 'C:/Users/10188/Desktop/기업가치/상관분석/기업가치분석_data_result'
dir_master = os.path.join(dir_root, '01. data', 'master')
dir_processed = os.path.join(dir_root, '01. data', 'processed')
dir_result = os.path.join(dir_root, '03. result')

In [21]:
y = '유동비율(비율)'
c1 = 'Accenture'
c2 = 'Infosys'
c3 = 'TCS'
c4 = 'Cognizant'

In [22]:
outperform = [c1, c2, c3]
lowperform = [c4]

In [23]:
target_out = [s+'_show' for s in outperform]
target_low = [s+'_show' for s in lowperform]
target_col = [s+'_show' for s in outperform+lowperform]

In [24]:
dir_result_all = os.path.join(dir_result, 'All_ITS', y)

In [25]:
df = pd.read_excel(os.path.join(dir_result_all, 'corr_y_x_over7.xlsx'))

In [26]:
df.columns

Index(['variable', 'group', 'importance', 'Accenture_filtered',
       'Infosys_filtered', 'TCS_filtered', 'Cognizant_filtered',
       'Accenture_show', 'Infosys_show', 'TCS_show', 'Cognizant_show'],
      dtype='object')

# Make index

In [27]:
df['Outperform_avg'] = df[list(set(target_out)&set(df.columns))].mean(axis=1)
df['Gap(Out~Low)'] = np.abs(df['Outperform_avg']-df[target_low[0]])
df['Std(Out)'] = df[list(set(target_out)&set(df.columns))].std(axis=1)

In [28]:
scaler = MinMaxScaler()
df['Gap normed+1'] = pd.DataFrame(scaler.fit_transform(df[['Gap(Out~Low)']])+1)[0]

In [29]:
scaler = MinMaxScaler()
df['Std normed+1'] = pd.DataFrame(scaler.fit_transform(df[['Std(Out)']])+1)[0]

In [30]:
df['index 1'] = df['Gap normed+1']/df['Std normed+1']

In [31]:
scaler = MinMaxScaler()
df['Std normed original'] = pd.DataFrame(scaler.fit_transform(df[['Std(Out)']]))[0]

In [32]:
df['Std normed 0.5'] = df['Std normed original']*0.5 + 0.5
df['1/Std normed'] = 1/df['Std normed 0.5']

In [33]:
df['Index 2'] = 0.6*df['Gap normed+1']+0.4*df['1/Std normed']

In [34]:
df[[s+'_marked' for s in outperform+lowperform]+[s+'_plot' for s in outperform+lowperform]] = np.nan

In [35]:
df.columns

Index(['variable', 'group', 'importance', 'Accenture_filtered',
       'Infosys_filtered', 'TCS_filtered', 'Cognizant_filtered',
       'Accenture_show', 'Infosys_show', 'TCS_show', 'Cognizant_show',
       'Outperform_avg', 'Gap(Out~Low)', 'Std(Out)', 'Gap normed+1',
       'Std normed+1', 'index 1', 'Std normed original', 'Std normed 0.5',
       '1/Std normed', 'Index 2', 'Accenture_marked', 'Infosys_marked',
       'TCS_marked', 'Cognizant_marked', 'Accenture_plot', 'Infosys_plot',
       'TCS_plot', 'Cognizant_plot'],
      dtype='object')

In [36]:
writer = pd.ExcelWriter(os.path.join(dir_result_all, 'correlation_index.xlsx'), engine='xlsxwriter')
df.to_excel(writer, index=False)
writer.save()